# 📋 Documentación del Script de Limpieza de Datos
## Análisis de RCP Transtelefónica

---

Este notebook documenta detalladamente el proceso de limpieza y preparación de datos para el análisis científico de la **Reanimación Cardiopulmonar (RCP) Transtelefónica**.

### 🎯 **Objetivo del Análisis**
Evaluar si la RCP guiada por teléfono mejora:
- El **ROSC** (Retorno de Circulación Espontánea)
- La **supervivencia a 7 días** tras una parada cardiaca extrahospitalaria

### 📊 **¿Qué hace este script?**
El script `cleandata.py` toma los datos crudos del hospital y los convierte en un dataset limpio y analizable, realizando múltiples transformaciones y cálculos automáticos.

## 🔄 **Flujo General del Procesamiento**

```
📁 Datos Crudos (Excel/CSV)
        ⬇️
🔧 1. Lectura y Renombrado de Columnas
        ⬇️
🔗 2. Combinación de Datos SVB y SVA
        ⬇️
⚙️ 3. Cálculo de Variables Derivadas
        ⬇️
🔢 4. Conversión de Variables Booleanas
        ⬇️
👥 5. Clasificación de Respondientes y ROSC
        ⬇️
🚫 6. Filtrado de Casos Traumáticos
        ⬇️
📞 7. Filtrado de Datos RCP Transtelefónica
        ⬇️
⏱️ 8. Cálculo de Tiempos de RCP
        ⬇️
🏥 9. Determinación de Supervivencia y CPC
        ⬇️
📋 10. Organización Final y Guardado
        ⬇️
📊 Datos Listos para Análisis
```

## 📖 **Explicación Detallada de Cada Paso**

### **Paso 1: 📁 Lectura y Renombrado de Columnas**

**¿Qué hace?**
- Lee el archivo CSV con los datos hospitalarios crudos
- Cambia los nombres de las columnas a nombres más claros y estándar

**¿Por qué es importante?**
- Los datos originales tienen nombres complicados como "NUM INFORME" → se convierte en "n_informe"
- Facilita el trabajo posterior con nombres consistentes

**Ejemplo de transformación:**
```
ANTES:          DESPUÉS:
NUM INFORME  →  n_informe
EDAD         →  edad
SEXO         →  sexo
ROSC         →  rosc
```

### **Paso 2: 🔗 Combinación de Datos SVB y SVA**

**¿Qué significa SVB y SVA?**
- **SVB**: Soporte Vital Básico (ambulancia básica)
- **SVA**: Soporte Vital Avanzado (ambulancia con médico)

**¿Qué hace este paso?**
- Combina la información de ambos tipos de ambulancia cuando atienden al mismo paciente
- Suma los tiempos de ambas unidades para tener el tiempo total de intervención

**¿Por qué es importante?**
- A veces un paciente es atendido primero por SVB y luego por SVA
- Necesitamos la información completa para el análisis
- **CRÍTICO**: Esto debe hacerse ANTES de eliminar casos, para no perder información

### **Paso 3: ⚙️ Cálculo de Variables Derivadas**

**¿Qué hace?**
Calcula nuevas variables importantes a partir de los datos originales:

#### **A) Tiempo de Llegada de la Unidad**
- Suma los tiempos desde la llamada hasta que llega la ambulancia
- **Fórmula**: C0+C1+C2+C3 (diferentes fases del proceso)

#### **B) Clasificación de Ritmo Cardíaco**
- **Ritmo Desfibrilable** (1): Fibrilación Ventricular, Taquicardia Ventricular
- **Ritmo No Desfibrilable** (0): Asistolia, Actividad Eléctrica sin Pulso

**¿Por qué es importante?**
- Los ritmos desfibrilables tienen mejor pronóstico
- El tiempo de llegada afecta la supervivencia

#### **C) Conversión de Edad a Números**
- Asegura que la edad esté en formato numérico para cálculos estadísticos

### **Paso 4: 🔢 Conversión de Variables Booleanas**

**¿Qué hace?**
Convierte texto a números para facilitar el análisis:

```
ANTES:              DESPUÉS:
"Verdadero"      →      1
"Falso"          →      0
"True"           →      1
""               →      0
```

**Variables afectadas:**
- `rcp_transtelefonica`: ¿Recibió instrucciones por teléfono?
- `desa_externo`: ¿Se usó un desfibrilador externo?
- `rcp_testigos`: ¿Los testigos hicieron RCP?

**¿Por qué es importante?**
- Los programas estadísticos trabajan mejor con números (0 y 1) que con texto
- Permite calcular porcentajes y hacer comparaciones fácilmente

### **Paso 5: 👥 Clasificación de Respondientes y ROSC**

**¿Qué hace?**
Analiza los textos médicos para identificar con precisión el tipo de respondiente utilizando **códigos de emergencia y términos específicos**:

#### **A) Sistema de Clasificación Mejorado:**

**👮 Policía:**
- **Códigos**: 091 (Policía Nacional), 092 (Policía Local), 062 (Guardia Civil)
- **Términos**: "policia", "municipal"

**🚒 Bomberos:**
- **Códigos**: 080 (Bomberos)
- **Términos**: "bombero", "beta" (unidades especiales)

**🏥 Sanitarios:**
- **Códigos**: 061 (SAMUR/Emergencias), 060 (Urgencias Sanitarias)
- **Servicios**: SUMMA (Urgencias Médicas), SAMUR (Servicio de Asistencia Municipal), SVB (Soporte Vital Básico)
- **Términos**: "sanitario", "medico", "enfermero", "tes", "hospital", "personal sanitario", "personal medico"

**👨‍👩‍👧‍👦 Legos:**
- **Términos**: "lego", "ciudadano", "testigo" (ciudadanos sin formación médica)

#### **B) Sistema de Priorización:**
1. **Sanitario** (mayor prioridad - personal médico especializado)
2. **Bombero** (formación en emergencias)
3. **Policía** (formación básica en primeros auxilios)
4. **Lego** (menor prioridad - ciudadanos comunes)

#### **C) Detección de ROSC:**
**ROSC** = Retorno de Circulación Espontánea (el corazón vuelve a latir)

**¿Cómo lo detecta?**
- Busca palabras como "ROSC", "recuperada" en los textos médicos
- Si hay tiempo registrado en C3_C4, también indica ROSC

**¿Por qué es importante esta clasificación detallada?**
- **Precisión científica**: Diferentes niveles de entrenamiento afectan la calidad de la RCP
- **Análisis estratificado**: Permite evaluar efectividad por tipo de respondiente
- **Políticas públicas**: Informa sobre qué grupos necesitan más entrenamiento
- **Validación de la RCP transtelefónica**: Evalúa si es más efectiva con ciertos tipos de respondientes

### **Paso 6: 🚫 Filtrado de Casos Traumáticos**

**¿Qué hace?**
Elimina casos que NO son paradas cardiacas médicas:

**Casos eliminados:**
- Trauma, heridas
- Caídas, accidentes
- Ahogamientos
- Suicidios
- Precipitados (caídas desde altura)

**¿Por qué es importante?**
- Este estudio se centra en paradas cardiacas **médicas** (problemas del corazón)
- Los casos traumáticos tienen causas y pronósticos muy diferentes
- La RCP transtelefónica puede no ser tan efectiva en trauma

**¿Cómo funciona?**
- Busca palabras clave en el texto de "consulta"
- Si encuentra estas palabras, elimina el caso del análisis

### **Paso 7: 📞 Filtrado de Datos RCP Transtelefónica**

**¿Qué hace?**
Asegura la calidad y consistencia de los datos relacionados con la RCP transtelefónica:

**Criterios de Inclusión:**
- Casos con información **clara y documentada** sobre RCP transtelefónica
- Tanto casos que **SÍ recibieron** RCP transtelefónica (valor = 1)
- Como casos que **NO recibieron** RCP transtelefónica (valor = 0)

**Criterios de Exclusión:**
- Casos con información **ambigua o contradictoria**
- Registros donde no se puede determinar con certeza si hubo o no RCP transtelefónica
- Datos incompletos que comprometan la validez del análisis

**¿Por qué es importante?**
- **Validez científica**: Para comparar grupos, necesitamos estar seguros de la clasificación
- **Reducción de sesgos**: Elimina casos que podrían distorsionar los resultados
- **Robustez estadística**: Mejora la confiabilidad de las conclusiones

**¿Cómo funciona?**
- Revisa la consistencia de la codificación booleana (0/1)
- Identifica y maneja casos con información conflictiva
- Mantiene solo los casos con clasificación confiable

**Momento crítico:** Este filtro debe aplicarse **DESPUÉS** de la combinación SVB/SVA y **ANTES** de los cálculos de tiempo, para asegurar que trabajamos solo con datos de alta calidad.

### **Paso 8: ⏱️ Cálculo de Tiempos de RCP**

**¿Qué hace?**
Calcula cuánto tiempo se realizó RCP utilizando **inteligencia artificial para leer texto médico**.

#### **Métodos de Detección:**

**A) Búsqueda Directa de Tiempo:**
- Busca frases como: "RCP 20 minutos", "reanimación 15 min"
- Convierte minutos a segundos automáticamente

**B) Cálculo por Horas de Inicio y Fin:**
- Toma la hora de la llamada
- Busca la hora de "exitus" (fallecimiento)
- Calcula la diferencia como tiempo de RCP

**C) Ajustes Inteligentes:**
- Si no había testigos haciendo RCP, suma el tiempo de llegada de la ambulancia
- Valida que los tiempos sean razonables (máximo 3 horas)

**¿Por qué es importante?**
- El tiempo de RCP afecta las posibilidades de supervivencia
- Permite comparar la duración de RCP entre grupos
- Muchos casos no tenían esta información calculada

### **Paso 9: 🏥 Determinación de Supervivencia y CPC**

**¿Qué hace?**
Analiza textos médicos complejos para determinar dos resultados críticos:

#### **A) Supervivencia a 7 Días:**
**¿Vivió o murió el paciente a los 7 días?**

**Indicadores de SUPERVIVENCIA:**
- "alta", "domicilio", "recuperado"
- "consciente", "despierta", "vigil"
- "estable", "vive"

**Indicadores de FALLECIMIENTO:**
- "exitus", "fallecido", "muerte"
- "óbito", "defunción", "fin"

#### **B) CPC (Cerebral Performance Category):**
**Escala neurológica de 1-5:**

- **CPC 1**: Buen rendimiento neurológico (consciente, orientado)
- **CPC 2**: Discapacidad moderada (confuso, desorientado)
- **CPC 3**: Discapacidad severa (dependiente para actividades básicas)
- **CPC 4**: Estado vegetativo (no responde, en coma)
- **CPC 5**: Muerte cerebral o muerte

**¿Cómo funciona?**
1. Busca valores CPC explícitos en el texto
2. Si no los encuentra, infiere el CPC basándose en descripciones neurológicas
3. **Regla de consistencia**: Si CPC=5 → Supervivencia=0

**¿Por qué es importante?**
- No basta con sobrevivir, importa la calidad de vida neurológica
- Es el estándar internacional para evaluar resultados post-RCP

### **Paso 10: 📋 Organización Final y Guardado**

**¿Qué hace?**
- Selecciona solo las columnas importantes para el análisis
- Las ordena en un formato lógico
- Guarda el dataset limpio como `cleaned_data.csv`

**Columnas finales del dataset:**
1. `n_informe`: Identificador del caso
2. `fecha`: Fecha y hora de la llamada
3. `edad`: Edad del paciente
4. `sexo`: Sexo del paciente
5. `rcp_transtelefonica`: RCP guiada por teléfono (0/1)
6. `rcp_testigos`: RCP por testigos (0/1)
7. `respondiente_rcp`: Tipo de persona que hizo RCP
8. `desa_externo`: Uso de desfibrilador externo (0/1)
9. `ritmo`: Ritmo desfibrilable (0/1)
10. `tiempo_llegada_unidad`: Tiempo hasta llegada ambulancia (segundos)
11. `tiempo_rcp`: Duración de la RCP (segundos)
12. `rosc`: Retorno circulación espontánea (0/1)
13. `supervivencia_7dias`: Supervivencia a 7 días (0/1)
14. `cpc`: Escala neurológica (1-5)

**¿Por qué es importante?**
- Elimina columnas temporales que ya no se necesitan
- Crea un dataset "limpio" listo para análisis estadístico
- Facilita el trabajo de otros investigadores

## 📊 **Resumen Estadístico Automático**

**¿Qué hace?**
Al final del procesamiento, el script genera automáticamente un resumen estadístico completo que incluye:

### **📈 Estadísticas Descriptivas:**
- **Demografía**: Edad promedio, distribución por sexo
- **Intervenciones**: % con RCP transtelefónica, % con RCP de testigos
- **Características clínicas**: % ritmos desfibrilables, tiempos promedio
- **Resultados**: % ROSC, % supervivencia, distribución CPC

### **🔍 Análisis Cruzado Clave:**
- **ROSC con vs sin RCP transtelefónica**
- **Supervivencia con vs sin RCP transtelefónica**
- **ROSC por tipo de ritmo cardíaco**

**¿Por qué es útil?**
- Proporciona una visión inmediata de los datos procesados
- Permite detectar problemas o patrones inesperados
- Facilita la interpretación inicial de los resultados
- Sirve como base para el análisis estadístico posterior

## 🎯 **Resultados Esperados del Análisis**

Con los datos procesados por este script, podremos responder preguntas científicas como:

### **Preguntas Principales:**
1. **¿La RCP transtelefónica mejora el ROSC?**
2. **¿La RCP transtelefónica mejora la supervivencia a 7 días?**
3. **¿La RCP transtelefónica mejora el resultado neurológico (CPC)?**

### **Análisis Secundarios:**
- Efecto de la edad y sexo en los resultados
- Importancia del tiempo de llegada de la ambulancia
- Diferencias entre tipos de respondientes
- Impacto del ritmo cardíaco inicial
- Efecto de la duración de la RCP

### **Métodos Estadísticos Posteriores:**
- **Regresión logística multivariante**: Para ajustar por variables de confusión
- **Pruebas de chi-cuadrado**: Para comparar proporciones
- **Curvas ROC**: Para evaluar capacidad predictiva
- **Forest plots**: Para visualizar odds ratios
- **Análisis de supervivencia**: Para estudiar tiempo hasta eventos

## 🔧 **Aspectos Técnicos Importantes**

### **🤖 Procesamiento de Lenguaje Natural:**
- El script utiliza **expresiones regulares** (regex) para buscar patrones en texto médico
- Puede manejar variaciones en la escritura médica ("RCP", "reanimación", "masaje cardíaco")
- Es tolerante a errores tipográficos comunes

### **✅ Control de Calidad:**
- **Validación de rangos**: Los tiempos no pueden ser negativos o excesivamente largos
- **Consistencia lógica**: Si CPC=5 (muerte), entonces supervivencia=0
- **Manejo de datos faltantes**: Convierte valores vacíos a 0 cuando es apropiado

### **⚠️ Limitaciones:**
- Depende de la calidad del texto médico original
- Algunos casos pueden requerir revisión manual
- Las inferencias de CPC pueden no ser 100% precisas

### **📂 Archivos Generados:**
- **`data/cleaned_data.csv`**: Dataset principal para análisis
- **Resumen en terminal**: Estadísticas descriptivas inmediatas
- **Log de procesamiento**: Información sobre casos procesados

## 🚀 **Cómo Ejecutar el Script**

### **Requisitos:**
1. Python 3.x instalado
2. Librería pandas instalada
3. Archivo `data/rawdata_2year.csv` con los datos originales

### **Ejecución:**
```bash
# Desde la terminal, en el directorio del proyecto:
python cleandata.py
```

### **Lo que verás:**
1. Mensaje de confirmación: "Cleaned data saved to data/cleaned_data.csv"
2. Resumen estadístico completo en la terminal
3. Archivo CSV listo para análisis en la carpeta `data/`

### **Tiempo de Ejecución:**
- Típicamente 30-60 segundos para ~600-800 casos
- Depende del tamaño del dataset y la complejidad del texto médico

## 📚 **Conclusión**

Este script de limpieza de datos es una herramienta sofisticada que:

### **✅ Automatiza tareas complejas:**
- Lectura e interpretación de texto médico
- Cálculos de tiempos y clasificaciones
- Inferencia de resultados clínicos

### **🎯 Prepara datos para análisis científico riguroso:**
- Dataset estandarizado y consistente
- Variables codificadas apropiadamente
- Control de calidad integrado

### **📊 Facilita la investigación médica:**
- Reduce tiempo de preparación de datos de semanas a minutos
- Minimiza errores humanos en la codificación
- Permite reproducibilidad del análisis

**El resultado final es un dataset limpio, estandarizado y listo para responder preguntas científicas importantes sobre la efectividad de la RCP transtelefónica en la supervivencia de pacientes con parada cardiaca extrahospitalaria.**